# Анализ базы данных кино

## 1. Постановка задачи

Собрана база данных кино. В базе присутствуют столбцы с id, названия, дату релиза, описание, показы, средняя оценка, количество оценок и наличие видео. Всего собрано 9463 записи.

**Студии, которые могут стать заказчиками анализа:**
- Walt Disney Company
- Базелевс
- Yellow, Black and White Group
- Paramount Pictures
- Кинокомпания СТВ

**Цели анализа:**
1. Выявить среднюю разницу оценки первой и последующих частей фильмов
2. Определить, какие слова чаще встречаются в описании фильмов с высокой оценкой
3. Найти наиболее удачные года кинопроката
4. Обнаружить закономерности дат выпуска относительно среднего успеха фильмов

In [ ]:
# 2. Паспорт датасета (Data Understanding)
import pandas as pd
df = pd.read_csv('dataset/Movie-Dataset-Latest.csv')
print('Размерность (строк, столбцов): ', df.shape)
print('Типы всех столлбцов: ')
df.info()

In [ ]:
# 3. Аудит качества данных
df.info()